In [ ]:
##### Llama-index 是构建上下文增强LLM应用的框架
# LlamaIndex 对您如何使用 LLM 没有限制。您可以将 LLM 用作自动完成、聊天机器人、代理等等。它只是让使用它们变得更容易。我们提供的工具包括：
# - 数据连接器从其原生源和格式摄取您的现有数据。这些可以是 API、PDF、SQL 以及（更多）其他来源。
# - 数据索引以易于 LLM 消费且性能良好的中间表示形式来组织您的数据。
# - 引擎提供对您数据的自然语言访问。例如
#   - 查询引擎是强大的问答界面（例如，一个 RAG 流程）。
#   - 聊天引擎是用于与您的数据进行多消息、“来回”交互的对话界面。
# - 代理是 LLM 驱动的知识工作者，并通过工具进行增强，这些工具从简单的辅助函数到 API 集成等等。
# - 可观测性/评估集成使您能够在良性循环中严格地实验、评估和监控您的应用程序。
# - 工作流允许您将以上所有功能组合到一个事件驱动系统中，这比其他基于图的方法更加灵活。


#### 大语言模型（LLMs）
# LLMs 是 LlamaIndex 诞生的根本创新。它们是一种人工智能 (AI) 计算机系统，能够理解、生成和处理自然语言，
# 包括根据其训练数据或在查询时提供给它们的数据回答问题。


#### 代理应用
# 当 LLM 在应用程序中使用时，它通常用于做出决策、采取行动和/或与世界交互。这是代理应用的核心定义。
# 尽管代理应用的定义很广泛，但有几个关键特征定义了代理应用
## LLM 增强：LLM 通过工具（即代码中任意可调用的函数）、内存和/或动态提示进行增强。
## 提示链：使用多个相互构建的 LLM 调用，一个 LLM 调用的输出用作下一个调用的输入。
## 路由：LLM 用于将应用程序路由到应用程序中的下一个适当的步骤或状态。
## 并行性：应用程序可以并行执行多个步骤或操作。
## 编排：使用 LLM 的层级结构来编排较低级别的操作和 LLM。
## 反思：LLM 用于反思和验证前一步骤或 LLM 调用的输出，这可以用来指导应用程序进入下一个适当的步骤或状态。
#  在 LlamaIndex 中，您可以使用 Workflow 类来编排一系列步骤和 LLMs，从而构建代理应用


#### 代理
# 我们将代理定义为“代理应用”的一个具体实例。
# 代理是一种软件，通过将 LLMs 与其他工具和内存结合，在推理循环中自主地执行任务，该循环决定接下来使用哪个工具（如果需要）。
# 这在实践中意味着：
# - 代理接收用户消息 
# - 代理使用 LLM，结合先前的聊天历史、工具和最新的用户消息来确定要采取的下一个适当行动 
# - 代理可能会调用一个或多个工具来协助处理用户的请求 
# - 如果使用了工具，代理将解释工具输出并用其指导下一个行动 
# - 一旦代理停止采取行动，它会将最终输出返回给用户


#### 使用案例
# 数据支持的 LLM 应用有无数的使用案例，但大致可以分为四类
# - 代理：代理是由 LLM 驱动的自动化决策器，通过一套工具与世界交互。
#        代理可以执行任意数量的步骤来完成给定任务，动态决定最佳行动方案，而不是遵循预设步骤。
#        这使其具有额外的灵活性来处理更复杂的任务。
# - 工作流：LlamaIndex 中的工作流是一种特定的事件驱动抽象，允许您编排一系列步骤和 LLMs 调用。
#          工作流可用于实现任何代理应用，是 LlamaIndex 的核心组件。
# - 结构化数据提取： Pydantic 提取器允许您指定要从数据中提取的精确数据结构，并以类型安全的方式使用 LLMs 填充缺失的部分。
#                这对于从 PDF、网站等非结构化源中提取结构化数据非常有用，也是自动化工作流的关键。
# - 查询引擎：查询引擎是一个端到端流程，允许您对数据提出问题。它接收自然语言查询，并返回响应以及检索到的并传递给 LLM 的参考上下文。
# - 聊天引擎：聊天引擎是一个端到端流程，用于与您的数据进行对话（多次往返而不是单一问答）。



#### 1. 构建Agent  代理

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent 
# FunctionAgent
# 创建工具函数  func_tools [func_a, func_b, func_c]
# 初始化llm      llm 
# 初始化Agent   workflow=FunctionAgent(llm tools prompt ...)
# # 提问   workflow.run(query, context)



### 工具集 Llama-Cloud
# from llama_index.tools.xx import xxx



### 维护状态
# 默认情况下，AgentWorkflow 在各次运行之间是无状态的。这意味着 Agent 不会记住先前的运行信息。
# 为了维护状态，我们需要跟踪先前的状态。
# 在 LlamaIndex 中，Workflow 有一个 Context 类，可用于在运行内部和运行之间维护状态。
# 由于 AgentWorkflow 只是一个预构建的 Workflow，我们现在也可以使用它。
from llama_index.core.workflow import Context 
# context = Context(workflow)
### 维护更长的状态
# Context 是可序列化的，因此可以保存到数据库、文件等位置，稍后重新加载。
# JsonSerializer 是一个简单的序列化器，它使用 json.dumps 和 json.loads 来序列化和反序列化 Context。
# JsonPickleSerializer 是一个使用 pickle 来序列化和反序列化 Context 的序列化器。如果你的 Context 中包含不可序列化的对象，可以使用此序列化器。
from llama_index.core.workflow import JsonPickleSerializer, JsonSerializer
# Context 序列化为字典并保存到文件中
# cdx_dict = context.to_dict(serializer=JsonSerializer())
# restored_ctx = Context.from_dict(workflow, cdx_dict, serializer=JsonSerializer())
Context.get()
### 工具与状态
# 工具也可以定义为可以访问 Workflow Context。这意味着你可以从 Context 中设置和检索变量并在工具中使用它们，或者在工具之间传递信息。
# AgentWorkflow 使用一个名为 state 的 Context 变量，该变量对每个 Agent 都可用。你可以依赖 state 中的信息而无需显式传入。
from llama_index.core.agent.workflow import AgentWorkflow 
"""
# 要访问 Context，Context 参数应该是工具的第一个参数，就像我们在这里所做的，在一个简单地向状态添加名称的工具中
def set_name(ctx: Context, name: str) -> str:
    state = ctx.store.get("state")
    state["name"] = name 
    ctx.set("state", state)

# 把普通的 Python 函数包装成 可被 LLM 调用的工具
workflow = AgentWorkflow.from_tools_or_functions([set_name], 
                                                 llm=llm, 
                                                 system_prompt="xxx",
                                                 initial_state={"name":"unset"})
                                                 
就是说 workflow_1  就是将变量x变成 123
再套一个 workflow = AgentWorkflow.from_tools_or_functions(..) # 变量x变成了什么xx
"""

#### 流式输出和事件 
# 在实际应用中，代理可能需要很长时间才能运行。向用户提供关于代理进度的反馈至关重要，而流式传输可以实现这一点。
# AgentWorkflow 提供了一系列预构建的事件，你可以使用它们向用户流式传输输出。让我们看看如何做到这一点。
# 首先，我们将介绍一个需要一些时间执行的新工具。在这种情况下，我们将使用一个名为 Tavily 的网络搜索工具，它在 LlamaHub 中可用。
# llama-index-tools-tavily-research  pypi
# 它需要一个 API 密钥，我们将在 .env 文件中将其设置为 TAVILY_API_KEY 
from llama_index.tools.tavily_research import TavilyToolSpec
import os
# 初始化工具
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))
# 初始化代理
workflow = FunctionAgent(tools=tavily_tool.to_tool_list(),
                         ) # llm, system_prompts 
# 在之前的示例中，我们使用 await 在 workflow.run 方法上获取代理的最终响应。
# 然而，如果我们不对响应进行等待，我们将获得一个异步迭代器，我们可以迭代该迭代器来获取传入的事件。这个迭代器将返回各种事件。
# 我们将从一个 AgentStream 事件开始，它包含输出传入时的“增量”（最新的变化）。我们需要导入该事件类型
from llama_index.core.agent.workflow import AgentStream
handler = workflow.run(user_msg="xxxxx")
async for event in handler.stream_events():
    if isinstance(event, AgentStream):
        print(event.delta, end="", flush=True)
# AgentStream 只是 AgentWorkflow 运行时发出的众多事件之一。其他的事件包括：
# - AgentInput    : 开始代理执行的完整消息对象
# - AgentOutput   : 来自代理的响应
# - ToolCall      : 调用了哪些工具以及使用了哪些参数
# - ToolCallResult: 工具调用的结果 


#### 人在回路中
# 工具也可以被定义为引入人在回路中。这对于需要人工输入的任务很有用，例如确认工具调用或提供反馈。
# AgentWorkflow 的底层工作方式是通过运行既发出又接收事件的步骤来实现。
# 为了引入人在回路中，我们将让工具发出一个工作流中其他任何步骤都不接收的事件。
# 然后，我们将告诉工具等待，直到接收到特定的“回复”事件。
# 我们有内置的 InputRequiredEvent 和 HumanResponseEvent 事件可用于此目的。
# 如果您想捕获不同形式的人工输入，可以对这些事件进行子类化以满足您的偏好。我们来导入它们。
from llama_index.core.workflow import (InputRequiredEvent, HumanResponseEvent)
# 接下来，我们将创建一个执行假定危险任务的工具。这里有几点新内容：
# - wait_for_event 用于等待 HumanResponseEvent。
# - waiter_event 是写入事件流的事件，用于告知调用者我们正在等待响应。
# - waiter_id 是此特定等待调用的唯一标识符。它有助于确保我们对于每个 waiter_id 只发送一个 waiter_event。
# - requirements 参数用于指定我们要等待具有特定 user_name 的 HumanResponseEvent。
async def dangerous_task(ctx: Context) -> str:
    # emit a waiter event (InputRequiredEvent)
    # and wait until we see a HumanResponseEvent
    question = " .. "
    response = await ctx.wait_for_event(
        HumanResponseEvent,
        waiter_id=question,
        waiter_event=InputRequiredEvent(
            prefix=question,
            user_name='Lauire',
        )
        requirements={'user_name': 'lauire'}
    )
    # act on input from the event
    if response.response.strip().lower == "yes":
        return "dangerous task completed successfully"
    else:
        return "dangerous task aborted"
# 创建Agent，传入dangerous_task
workflow = FunctionAgent(tools=[dangerous_task], ) # llm system_prompt
# 现在运行工作流，像处理任何其他流式事件一样处理InputRequiredEvent, 并使用Send_event方法传入一个HumanResponseEvent进行响应
handler = workflow.run(user_msg=" .. ")      # 可等待对象，同时还能流式产出中间事件 
async for event in handler.stream_events():
    # capture keyboard input
    response = input(event.prefix)
    # send or response back 
    handler.ctx.send_event(
        HumanResponseEvent(
            response=response,
            user_name=event.user_name,   
        )
    )
response = await handler
print(str(response))
# 您可以使用任何方式来捕获输入；可以使用 GUI、音频输入，甚至可以引入另一个独立的 Agent。
# 如果您的输入需要一段时间或在另一个进程中发生，您可能希望序列化上下文并将其保存到数据库或文件中，以便之后可以恢复工作流。


#### AgentWorkflow的多智能体系统
# AgentWorkflow 单个智能体、多智能体系统。 多智能体系统中多个智能体协作完成任务，并在需要时将控制权互相移交
# - ResearchAgent  : 它将搜索网络一查找给定主题的信息
# - WriteAgent     : 将用ResearchAgent检索到的信息来撰写报告
# - ReviewAgent    : 将审查报告并提供反馈
## 需要用到的工具
# web_search工具，  Tavily
# record_notes工具，将网络搜索道德研究保存到状态中（AgentWorkflow使用一个名为state的Context变量），然后其他工具就可以使用它
# write_repot工具，使用ResearchAgent检索到的信息撰写报告
# review_report工具，审查报告和提供反馈
# < 2025-09-07-morning >


#### 2.构建工作流

In [ ]:
### 什么是工作流？ 
# 工作流是一种事件驱动、基于步骤的方式，用于控制应用程序的执行流程。
# 您的应用程序被划分为称为“步骤”（Steps）的部分，这些步骤由“事件”（Events）触发，并且本身也会发出事件，从而触发进一步的步骤。
# 通过组合步骤和事件，您可以创建任意复杂、封装逻辑的流程，使您的应用程序更易于维护和理解。一个步骤可以是单行代码，也可以是复杂的智能体。
# 它们可以具有任意输入和输出，这些输入和输出通过事件传递。
### 为什么使用工作流 ？
# 随着生成式 AI 应用变得越来越复杂，管理数据流和控制应用程序执行变得越来越困难。
# 工作流提供了一种管理此复杂性的方法，将应用程序分解为更小、更易于管理的部分。
## 其他框架和 LlamaIndex 本身之前曾尝试使用有向无环图（DAG）解决此问题，但这些方法存在一些工作流没有的限制：
# - 循环和分支等逻辑需要编码到图的边缘中，这使得它们难以阅读和理解。
# - 在 DAG 的节点之间传递数据会产生可选值和默认值以及应传递哪些参数的复杂性。
# - 对于尝试开发复杂、循环、分支 AI 应用的开发者来说，DAG 并不自然。
# 
# 
#### 基本工作流
# 工作流内置于 LlamaIndex 核心中  
# from llama_index.core.workflow ..
# from llama_index.utils.workflow ..
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
# 
#### 单步工作流
# 工作流通常实现为一个继承自WorkFlow的类。该类可以定义为任意数量的步骤，每个步骤都是用 @step 装饰器修饰的方法：
class MyWorkflow(Workflow):
    @step 
    async def my_Step(self, ev: StartEvent) -> StopEvent:
        # do something 
        return StopEvent(result="hello world!")
# w = MyWorkflow(timeout=100, verbose=False)
# result = await w.run()
# print(result)
# 
# Start事件和Stop事件
# StartEvent 和 StopEvent 是用于启动和停止工作流的特殊事件。
# 任何接受 StartEvent 的步骤都会由 run 命令触发。
# 发出 StopEvent 将结束工作流的执行并返回最终结果，即使其他步骤尚未执行。
#
#
#### 在普通python中运行工作流
# 工作流默认是异步的，async   ，因此您可以使用 await 获取 run 命令的结果。这在 Notebook 环境中运行良好
# 在普通的 Python 脚本中，您需要导入 asyncio 并将代码包装在一个异步函数中，像这样
"""  .py import asyncio
import asyncio
async def main():
    w = MyWorkflow(timeout=100, verbose=False)
    result = await w.run()
    print(result)
"""
# 
# 
#### 可视化工作流  还不错
#  工作流的一个很棒的功能是内置的可视化工具，我们已经安装了它。让我们可视化刚刚创建的简单工作流
# from llama_index.utils.workflow import draw_all_possible_flows
# draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")
# 
# 
#### 自定义事件
# 通过定义可以由步骤发出并触发其他步骤的自定义事件来创建多个步骤。
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
)
from llama_index.utils.workflow import draw_all_possible_flows 
class FirstEvent(Event):
    first_output: str 
class SecondEvent(Event):
    second_output: str 
# step_one 接受一个 StartEvent 并返回一个 FirstEvent
# step_two 接受一个 FirstEvent 并返回一个 SecondEvent
# step_three 接受一个 SecondEvent 并返回一个 StopEvent
class MyWorkflowb(Workflow):
    @step
    async def step_one(self, ev: StartEvent) -> FirstEvent:
        print(ev.first_input)
        return FirstEvent(first_output="first step complete.")    
    
    @step
    async def step_two(self, ev:FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output="second step complete.")
    
    @step 
    async def step_three(self, ev:SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="Worflow complete.")
    
# w = MyWorkflowb(timeout=10, verbose=False)
# result = await w.run(first_input="start the workflow.")
# print(result)
# #  hello world!
# # start the workflow.
# # first step complete.
# # second step complete.
# # Worflow complete.
# from llama_index.utils.workflow import draw_all_possible_flows
# draw_all_possible_flows(MyWorkflowb, filename="multi_step_workflow.html")
# 
# 
#### 分支和循环
# 工作流的一个关键特性是它们能够实现分支和循环逻辑，比基于图的方法更简单、更灵活。
# 
### 工作流中的循环 
# 要创建一个循环，我们将使用前一个教程中的示例 `MyWorkflow` 并添加一个新的自定义事件类型。
# 我们将它命名为 `LoopEvent`，但同样它可以是任何任意名称。
import random 
class LoopEvent(Event):
    loop_output: str 

class MyWorkflowc(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent | LoopEvent:
        if random.randint(0,1) == 1:
            print("bad thing happened.")
            return LoopEvent(loop_output="back to step one.")
        else:
            print("good thing happened.")
            return FirstEvent(first_output="first step complete.")  
    
    @step
    async def step_two(self, ev:FirstEvent) -> SecondEvent:
        print(ev.first_output)
        return SecondEvent(second_output="second step complete.")
    
    @step 
    async def step_three(self, ev:SecondEvent) -> StopEvent:
        print(ev.second_output)
        return StopEvent(result="Worflow complete.")
# w = MyWorkflowb(timeout=10, verbose=False)
# result = await w.run(first_input="start the workflow.")
# print(result)
# from llama_index.utils.workflow import draw_all_possible_flows
# draw_all_possible_flows(MyWorkflowc, filename="multi_step_loop_workflowc.html")
# 
# 
### 工作流中的分支 
# 与循环密切相关的是分支。正如您已经看到的，您可以有条件地返回不同的事件。让我们看一个分支到两个不同路径的工作流。
class BranchA1Event(Event):
    payload: str


class BranchA2Event(Event):
    payload: str


class BranchB1Event(Event):
    payload: str


class BranchB2Event(Event):
    payload: str


class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to branch B")
            return BranchB1Event(payload="Branch B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch A complete.")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch B complete.")
# w = BranchWorkflow(timeout=10, verbose=False)
# result = await w.run(first_input="start the workflow.")
# print(result)
# from llama_index.utils.workflow import draw_all_possible_flows
# draw_all_possible_flows(BranchWorkflow, filename="multi_step_branch_workflow.html")   
#
# 
#  
#### 维护状态
# 在目前位置的示例中，我们使用自定义事件的属性在步骤间传递数据，这是一种强大的数据传递方式，但也有局限性。
# 例如，我想在不直接相连的步骤之间传递数据，则需要通过中间的所有步骤来传递数据，这会让代码难写难以阅读和维护。
#  AgentWorkflow 的 state  ： Context  共享的。
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context,
)
# Start 事件，检查数据是否已加载到上下文中。如果还没有，返回一个SetupEvent，该事件除法setup函数加载数据并返回start
class SetupEvent(Event):
    query: str 
class StepTwoEvent(Event):
    query: str 

class StatefulFlow(Workflow):
    @step 
    async def start(
        self, ctx:Context, ev: StartEvent
    ) -> SetupEvent | StepTwoEvent:
        db = await ctx.store.get("some_database", default=None) 
        if db is None:
            print("need load data.")
            return SetupEvent(query=ev.query)
        # do something 
        return StepTwoEvent(query=ev.query)
    
    @step
    async def setup(self, ctx:Context, ev:SetupEvent) -> StartEvent:
        # load data 
        await ctx.store.set("some_database", [1,2,3])
        return StartEvent(query=ev.query)
    
    @step 
    async def step_two(self, ctx:Context, ev:StepTwoEvent) -> StopEvent:
        # do something there 
        print("Data is :", await ctx.store.get("some_database"))
        
        return StopEvent(result=await ctx.store.get("some_database"))
    
# w = StatefulFlow(timeout=10, verbose=False)
# result = await w.run(query="Some query")
# print(result)
# from llama_index.utils.workflow import draw_all_possible_flows
# draw_all_possible_flows(StatefulFlow, filename="multi_step_state_workflow.html")  
#
#
#### 流式事件
# 弘佐六可能很复杂， 它们旨在处理复杂、分支、并行的逻辑，这意味着它们可能需要一些时间才能完全执行。
# 为了给用户提供良好的体验，通常会希望通过流式事件来提供进度指示。工作流在Context对象上内置了对此的支持。
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)
from dotenv import load_dotenv
load_dotenv()
from llama_index.utils.workflow import draw_all_possible_flows

# local_chat_model
from llama_index.llms.huggingface import HuggingFaceLLM 
chat_model = HuggingFaceLLM(
    # cache_root可以省，model_name
    # or  local_dir
    model_name     = r"Qwen/Qwen3-1.7B",
    tokenizer_name = r"Qwen/Qwen3-1.7B",
    context_window = 3900,  
    max_new_tokens = 640,
    generate_kwargs={"temperature": 0.7, "top_k": 30, "top_p": 0.95},
    device_map     ='cpu' 
)


# 三步工作流，再加上一个事件来处理进行时的进度流式传输，
class FirstEvent(Event):
    first_output: str 
class SecondEvent(Event):
    second_output: str 
    response: str 
class ProgressEvent(Event):
    msg: str 
# 发送事件的工作流类
class MyWorkflowd(Workflow):
    @step
    async def step_one(self, ctx:Context, ev:StartEvent)->FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening."))
        return FirstEvent(first_output="first step copmplete")
    
    @step 
    async def step_two(self, ctx:Context, ev:FirstEvent)->SecondEvent:
        generator = await chat_model.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # allow the workflow to stream  this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="second step complete",
            response=str(response),
        )

    @step 
    async def step_three(self, ctx:Context, ev:SecondEvent)->StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="step three is happening"))
        return StopEvent(result="workflow complete.")

# w = MyWorkflowd(timeout=1000, verbose=False)
# handler = w.run(first_input = "Start the workflow")
# async for ev in handler.stream_events():
#     if isinstance(ev, ProgressEvent):
#         print(ev.msg)
#
# final_result = await handler
# print("final result:", final_result)
# draw_all_possible_flows(MyWorkflowd, filename="streaming_workflow.html")
# 
# 
# 
#### 并行执行
# 工作流还可以并行执行
# 当您有多个步骤可以独立运行且它们包含耗时的 await 操作时，并发执行非常有用，这允许其他步骤并行运行。
### 发射多个事件
# 一个步骤可以发射多个事件（之前的例子中都是返回一个事件），发射多个事件  end_event
class ParallelFlow(Workflow):
    @step 
    async def start(self, ctx:Context, ev:StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="q 1"))
        ctx.send_event(StepTwoEvent(query="q 2"))
        ctx.send_event(StepTwoEvent(query="q 3"))
    
    @step(num_workers=4)
    async def step_two(self, ctx:Context, ev:StepTwoEvent) -> StopEvent:
        print('Running slow query', ev.query)
        return StopEvent(result=ev.query)
# start 步骤发射 3 个 StepTwoEvent。step_two 步骤使用 num_workers=4 进行装饰，这会告诉工作流最多并发运行此步骤的 4 个实例（这是默认值）
# 执行，工作流会在第一个完成的查询之后停止。
#
### 收集事件
# 如果希望等待所有耗时操作完成后再继续下一步
# 以使用 collect_events 来实现这一点。
class StepThreeEvent(Event):
    query: str 
    
class ConcurrentFlow(Workflow):
    @step 
    async def start(self, ctx:Context, ev:StartEvent) -> StepTwoEvent:
        ctx.send_event(StepTwoEvent(query="q 1"))
        ctx.send_event(StepTwoEvent(query="q 2"))
        ctx.send_event(StepTwoEvent(query="q 3"))
    
    @step(num_wokers=4)
    async def ste_two(self, ctx:Context, ev:StepTwoEvent) -> StepThreeEvent:
        print("Running query ", ev.query)
        return StepThreeEvent(result=ev.query)
    
    @step
    async def step_three(self, ctx: Context, ev: StepThreeEvent) -> StopEvent:
        # wait until we receive 3 events
        result = ctx.collect_events(ev, [StepThreeEvent] * 3)
        if result is None:
            return None
        # do something with all 3 results together
        print(result)
        return StopEvent(result="Done")
# collect_events 方法位于 Context 上，它接受触发该步骤的事件以及要等待的事件类型数组。
# 在此示例中，我们正在等待 3 个相同 StepThreeEvent 类型的事件。
# step_three 步骤在每次收到 StepThreeEvent 时触发，但 collect_events 将返回 None，直到收到所有 3 个事件。
# 此时，该步骤将继续执行，您可以一起处理所有 3 个结果。
# 从 collect_events 返回的 result 是一个包含收集到的事件的数组，其顺序与收到事件的顺序一致。  
# 
# 
### 多种事件类型 
# 当然，您不必等待相同类型的事件。您可以等待任何您喜欢的事件组合，例如在此示例中
"""  
class ConcurrentFlow(Workflow):
    @step
    async def start(
        self, ctx: Context, ev: StartEvent
    ) -> StepAEvent | StepBEvent | StepCEvent:
        ctx.send_event(StepAEvent(query="Query 1"))
        ctx.send_event(StepBEvent(query="Query 2"))
        ctx.send_event(StepCEvent(query="Query 3"))

    @step
    async def step_a(self, ctx: Context, ev: StepAEvent) -> StepACompleteEvent:
        print("Doing something A-ish")
        return StepACompleteEvent(result=ev.query)

    @step
    async def step_b(self, ctx: Context, ev: StepBEvent) -> StepBCompleteEvent:
        print("Doing something B-ish")
        return StepBCompleteEvent(result=ev.query)

    @step
    async def step_c(self, ctx: Context, ev: StepCEvent) -> StepCCompleteEvent:
        print("Doing something C-ish")
        return StepCCompleteEvent(result=ev.query)

    @step
    async def step_three(
        self,
        ctx: Context,
        ev: StepACompleteEvent | StepBCompleteEvent | StepCCompleteEvent,
    ) -> StopEvent:
        print("Received event ", ev.result)

        # wait until we receive 3 events
        if (
            ctx.collect_events(
                ev,
                [StepCCompleteEvent, StepACompleteEvent, StepBCompleteEvent],
            )
            is None
        ):
            return None

        # do something with all 3 results together
        return StopEvent(result="Done")
"""
# 
# 
# 
#### 工作流子类化 <扩展工作流的方法 1>
# 工作流的另一个很棒的特性是它们的可扩展性。你可以使用其让人编写的工作流或LlamaIndex的内置工作流，并对其进行扩展、自定义。
# # 子类化  
# 工作流只是普通的Python类，这意味着你可以进行子类化以添加新功能。继承和多态
# 
# 
#### 嵌套工作流  <扩展工作流的方法 2>
# 扩展工作流的另一种方法是嵌套额外的工作流。可以在现有流程中创建明确的插槽，以便可以在其中提供一个完整的额外工作流。
# 例如：假设现在有一个使用LLM来反思该查询质量的查询。作者可能希望您修改反思步骤，并留下一个插槽来执行此操作
class Step2Event(Event):
    query: str 
class MainWorkflow(Workflow):
    @step 
    async def start(
        self, ctx:Context, ev:StartEvent, reflection_workflow: Workflow
    ) -> Step2Event:
        print("need add reflection")
        res = await reflection_workflow.run(query=ev.query)
    
    @step 
    async def step_two(self, ctx:Context, ev:Step2Event) -> StopEvent:
        print("query is ", ev.query)
        # do something 
        return StopEvent(result=ev.query)
# 需要完善 reflection_workflow
class ReflectionWorkflow(Workflow):
    @step 
    async def sub_start(Self, ctx:Context, ev:StartEvent) -> StopEvent:
        print("do reflection")
        return StopEvent(result="Improved query")
# 现在我们可以通过使用 add_worflows 方法提供这个自定义的反思嵌套流程来运行主工作流，
# 我们将一个 ReflectionWorkflow 类的实例传递给该方法
w = MainWorkflow(timeout=100, verbose=False)
w.add_workflows(reflection_workflow=ReflectionWorkflow())
print(result)
# 请注意，因为嵌套流程是一个完全不同的工作流，而不是一个步骤，所以
# draw_all_possible_flows  只会绘制  MainWorkflow  的流程图。
#### 默认工作流
# 函数的默认值一样的存在
# 
##### 可观测性
# # 可视化   draw_all_possible_flows 
# # 详细模式  verbose=True ,  简洁模式 verbose=Fasle
# # 步进执行  AgentWorkflow.run(stepwise=True) # 默认 False
# # 可视化最近的执行   
# from llama_index.utils.workflow import draw_most_recent_execution
# draw_most_recent_execution(w, filename="last_execution.html")
# 注意，这里传递的是工作流的实例 w，而不是类名。
# # 检查点
# 检查完整的工作流执行可能需要花费大量时间，而且通常只需要同时调试和观察几个步骤。
# 为了加快工作流开发周期，WorkflowCheckpointer会封装一个Workflow，并在每次运行的每个步骤完成后创建并存储checkpoint。
# 这些检查点可以被查看、检查，并被选作未来运行的起始点。
# from llama_index.core.workflow.checkpointer import WorkflowCheckpointer
"""  
from llama_index.core.workflow.checkpointer import WorkflowCheckpointer

w = ConcurrentFlow()
w_ckptr = WorkflowCheckpointer(workflow=w)

# run the workflow via w_ckptr to get checkpoints
handler = w_cptr.run()
await handler

# view checkpoints of the last run
w_ckptr.checkpoints[handler.run_id]

# run from a previous ckpt
ckpt = w_ckptr.checkpoints[handler.run_id][0]
handler = w_ckptr.run_from(checkpoint=ckpt)
await handler
"""
# 
# 
# # 第三方工具 Arize ...

#### 未绑定语法
# 从无绑定函数创建工作流
# 可以通过独立的或 无绑定 的函数定义工作流中的步骤，并使用 @step 装饰器将它们分配给工作流
# 
class TestWorkflow(Workflow):
    pass 
@step(Workflow=TestWorkflow)      # 配置一下装饰器就ok
def some_step(ev: StartEvent) -> StopEvent:
    return StopEvent()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Step one is happening.
 
I 
need 
the 
text 
to 
be 
in 
the 
first 
person 

perspective. 
I 
need 
the 
text 
to 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
I 
need 
the 
text 
to 
be 
in 
the 
first 
person 

perspective. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
text 
should 
be 
in 
the 
first 
person 

perspective. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
text 
should 
be 
in 
the 
first 
person 

perspective. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
text 
should 
be 
in 
the 
first 
person 

perspective. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
text 
should 
be 
in 
the 
first 
person 

perspective. 
The 
text 
should 
be 
in 
the 
style 
of 
the 
first 
chapter 
of 

Moby 

Dick. 
The 
te

#### 3.构建 RAG pipeline

- IngestionPipeline
- [IngestPipeline](https://docs.llamaindex.org.cn/en/stable/module_guides/loading/ingestion_pipeline/)  （LLM处理`准备好的数据`之前，需要准备好数据，准备好数据的过程`处理并加载数据`）
- 加载数据： 数据加载器、
- [转换数据](https://docs.llamaindex.org.cn/en/stable/module_guides/loading/ingestion_pipeline/transformations/)
- [索引与嵌入](https://docs.llamaindex.org.cn/en/stable/understanding/indexing/indexing/)和[存储数据](https://docs.llamaindex.org.cn/en/stable/understanding/storing/storing/)：
- 概念：[嵌入](https://docs.llamaindex.org.cn/en/stable/module_guides/models/embeddings/#list-of-supported-embeddings)

In [ ]:
###### 1 加载与提取
#
# IngestPipeline 1.加载数据 
# (加载数据、转换数据、索引和存储数据)     
#
###### 数据加载器   
# 在您选择的 LLM 处理您的数据之前，您需要加载数据。LlamaIndex 通过数据连接器来实现这一点，也称为 Reader。
# 数据连接器从不同的数据源摄取数据，并将数据格式化为 Document 对象。
# Document 是数据的集合（目前是文本，未来将包含图像和音频）以及关于该数据的元数据。
#
### SimpleDirectoryReader 可以将给定目录中的每个文件创建为文档
from llama_index.core import SimpleDirectoryReader      # md, pdf, docx,pptx, img, wav, mp4
# documents = SimpleDirectoryReader("./data").load_data()
### or --->  LlamaHub的读取器
from llama_index.core import download_loader
# from llama_index.readers.xxx import xxxx 
### 直接创建Document对象， Node
from llama_index.core import Document 
documents = Document(text="text")
#
# IngestPipeline 2.转换数据 
# (加载数据、转换数据、索引和存储数据)   
###### 转换
# 数据加载后，您需要在将其放入存储系统之前对其进行处理和转换。这些转换包括分块、提取元数据以及嵌入每个块。
# 这对于确保数据可以被检索并被 LLM 最佳地使用是必要的。
# 转换的输入/输出是 Node 对象（Document 是 Node 的子类）。转换也可以堆叠和重新排序。
# llma-index提供了用于转换文档的高级和低级 API。
""" 转换
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/ingestion_pipeline/transformations/
# 转换是一种以节点列表作为输入，并返回一个节点列表的事物。每个实现 Transformation 基类的组件都具有同步的 __call__() 定义和异步的 acall() 定义。
# 以下组件是Transformations的对象
- 文本分割器
- 节点解析器
- 元数据提取器
- Embeddings模型

"""
#
# ### 高级转换API 
#  .from_documents() 方法，它接受一个 Document 对象数组，并会正确解析和分块。然而，有时您可能需要更精细地控制文档如何分割。
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex.from_documents(documents)
vector_index.as_query_engine()
# 在底层，这将您的 Document 分割成 Node 对象，Node 对象与 Document 类似（它们包含文本和元数据），但与它们的父 Document 存在关系。
# 
# 如果您想通过这种抽象定制核心组件，例如文本分割器，您可以传入自定义的 transformations 列表或应用于全局 Settings。
from llama_index.core.node_parser import SentenceSplitter 
text_spliter = SentenceSplitter(chunk_size=512, chunk_overlap=128)
# global
from llama_index.core import Settings 
Settings.text_splitter = text_spliter
# pre-index
index = VectorStoreIndex.from_documents(
    documents=documents, transformations=[text_spliter]
)
#
### 低级转换API
# 显式定义这些步骤。
# 您可以通过将我们的转换模块（文本分割器、元数据提取器等）用作独立组件，或者在我们的声明式 转换管道接口 Ingestpipeline 中组合它们来完成此操作。
"""  IngestPipeline
# IngestionPipeline 使用了应用于输入数据的转换概念。这些转换应用于您的输入数据，生成的节点要么返回，要么插入到向量数据库（如果提供）。
# 每个节点+转换对都会被缓存，因此后续使用相同节点+转换组合的运行（如果缓存已持久化）可以使用缓存结果，从而节省时间。
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/ingestion_pipeline/

"""
#
## 将文档分割成node
# 处理文档的一个关键步骤是将其分割成“块”/Node 对象。核心思想是将您的数据处理成适合检索/馈送给 LLM 的小片段。
# LlamaIndex 支持多种 文本分割器，范围从基于段落/句子/token 的分割器到基于文件的分割器，如 HTML、JSON。
"""  文本分割器
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/node_parsers/modules/

"""
"""  这些也可以单独使用作为IngestPipeline的一部分
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/node_parsers/modules/#text-splitters

"""
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline 
from llama_index.core.node_parser import TokenTextSplitter 
documents = SimpleDirectoryReader("./data").load_data()
pipeline = IngestionPipeline(transformations=[TokenTextSplitter(), ...])
nodes = pipeline.run(documents=documents)
#
## 添加元数据
# 您还可以选择为文档和 Node 添加元数据。这可以手动完成，也可以使用自动元数据提取器完成。
""" 自动元数据提取器
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/documents_and_nodes/usage_metadata_extractor/"""
# 这里是关于 1) 如何定制 Document 和 2) 如何定制 Node 的指南。
"""  1) 如何定制 Document 
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/documents_and_nodes/usage_documents/
"""
"""  2) 如何定制 Node  
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/documents_and_nodes/usage_nodes/
"""
## 添加嵌入
# 要将 Node 插入向量索引，它应该具有嵌入。
""" IngestionPipeline or 嵌入 
https://docs.llamaindex.org.cn/en/stable/module_guides/loading/ingestion_pipeline/
https://docs.llamaindex.org.cn/en/stable/module_guides/models/embeddings/
"""
# 
## 直接创建和传递Node
from llama_index.core.schema import TextNode
node1 = TextNode(text='<text_chunk>', id='<node_id>')
node2 = TextNode(text='<text_chunk>', id='<node_id>')
index = VectorStoreIndex([node1, node2])



###### 2 索引与嵌入
#
#### 索引
# 加载数据后，您现在拥有一个 Document 对象列表（或 Node 对象列表）。现在是时候在这些对象之上构建一个 Index，以便您可以开始查询它们了。
"""什么是索引？
在 LlamaIndex 中，Index 是一个由 Document 对象构成的数据结构，旨在支持 LLM 进行查询。您的索引设计用于补充您的查询策略。
LlamaIndex 提供了几种不同的索引类型。这里我们将介绍其中两种最常见的。
- 向量存储索引
- 摘要索引
"""
#
### 向量存储索引
# VectorStoreIndex 是您将遇到的最常见的索引类型。
# 向量存储索引接收您的 Documents 并将其分割成 Nodes。然后，它为每个节点的文本创建向量嵌入，以便 LLM 进行查询。
#
### 摘要索引
# 摘要索引是一种更简单的索引形式，最适合于正如其名称所示，您正在尝试生成文档中文本摘要的查询。
# 它只是简单地存储所有 Documents 并将它们全部返回给您的查询引擎。



###### 3 存储  
# 加载并索引数据后，您可能希望存储它，以避免重新索引的时间和成本。默认情况下，您的索引数据仅存储在内存中。
#### 持久化到磁盘
# 存储索引数据最简单的方法是使用每个 Index 内置的 .persist() 方法，该方法将所有数据写入指定位置的磁盘。这适用于任何类型的索引。
index.storage_context.persist(persist_dir="<persist_dir>")
# 可组合图（Composable Graph）的示例
# graph.root_index.storage_context.persist(persist_dir="<persist_dir>")
# 可以通过持久化的碎银来避免重新加载和重新索引数据
from llama_index.core import StorageContext, load_index_from_storage
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="<persist_dir>")
# load index 
index = load_index_from_storage(storage_context)
# 重要提示：如果您使用自定义的 transformations、embed_model 等初始化了索引，则需要在 load_index_from_storage 期间传入相同的选项，或者将其设置为全局设置。
# from llama_index.core import Settings
# 
####  使用向量存储
# 正如在索引中所讨论的，Index 最常见的类型之一是 VectorStoreIndex。
# Embedding API调用 推荐。
# LlamaIndex 支持大量的向量存储，它们在架构、复杂性和成本方面各不相同。
# chroma 开源  支持向量存储
# 使用Chroma存储VectorStoreIndex中的embedding，需要
# - 初始化Chroma客户端
# - 在Chroma中创建一个Collection来存储数据
# - 在StorageContext中将Chroma指定为vector_store
# - 使用该StorageContext初始化VectorStoreIndex
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader 
from llama_index.vector_stores.chroma import ChromaVectorStore 
from llama_index.core import StorageContext 
# 
# load files 
documents = SimpleDirectoryReader("./xx").load_data()
# 
# initial chroma client, set local_db path  to persist
db = chromadb.PersistentClient(path="./chroma_db")
# 
# create collection
chroma_collection = db.get_or_create_collection("quickstart")
#
# assign chroma as the vector_store to the context 
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# 
# create index
index = VectorStoreIndex.from_documents(documents=documents, storage_context=storage_context)
# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("...")
print(response)
#
# 如果您已经创建并存储了 embedding，您会希望直接加载它们，而无需加载文档或创建新的 VectorStoreIndex
#### 加载
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
# init client
db = chromadb.PersistentClient(path="./load_chroma_db")
# get collection
chroma_collection = db.get_collection("quickstart")
# assign chroma as the vector_store to the context 
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# load index
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)
# create a query engine 
query_engine = index.as_query_engine()
response = query_engine.query("...")
print(response)
#
#### 插入文档或者节点
# 如果已经创建了索引，可以使用 insert 方法向索引中添加新文档。
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex([])
for doc in documents:
    index.insert(doc)
#
###### 4 查询 
# - 最简单的查询只是对 LLM 的一次提示调用：它可以是一个问题并获取答案，或者是一个摘要请求，亦或是一个更复杂的指令。
# - 更复杂的查询可能涉及重复/链式提示 + LLM 调用，甚至跨多个组件的推理循环。
# 所有查询的基础是 QueryEngine。获取 QueryEngine 的最简单方法是让您的索引为您创建一个
query_engine = index.as_query_engine()
response = query_engine.query("...")
print(response)
# 查询并不像初看起来那么简单。查询由三个不同的阶段组成:
# - 检索是指您从 Index 中找到并返回与您的查询最相关的文档。正如之前在索引中讨论的，最常见的检索类型是“top-k”语义检索，但还有许多其他检索策略。
#   - 后处理是指对检索到的 Node 可选地进行重排、转换或过滤，例如要求它们具有特定的元数据（如关键词）。
#   - 响应合成是指将您的查询、最相关的数据和提示组合起来，发送给您的 LLM 以返回响应。
#   - 提示
## LlamaIndex 提供了一个低级组合 API，让您可以精细控制您的查询。 
# 在此示例中，我们自定义检索器，对 top_k 使用不同的值，并添加一个后处理步骤，要求检索到的节点达到最低相似度分数才能包含在内。
# 这会使您在有相关结果时获得大量数据，但在没有任何相关内容时可能不会获得数据
# 还可以通过实现相应的接口来添加您自己的检索、响应合成和整体查询逻辑。
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
# build index
index = VectorStoreIndex.from_documents(documents)
# configure retriever 
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)
# configure response synthesizer 
response_synthesizer = get_response_synthesizer()
# assemble query engine 
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)
# query
response = query_engine.query("...")
print(response)
"""  检索其模块指南
https://docs.llamaindex.org.cn/en/stable/module_guides/querying/retriever/
"""
#
### 配置节点后处理器
# 我们支持高级 Node 过滤和增强，这可以进一步提高检索到的 Node 对象的相关性。这有助于减少时间/LLM 调用次数/成本或提高响应质量。
# - KeywordNodePostprocessor: 通过 required_keywords 和 exclude_keywords 过滤节点。
#   - SimilarityPostprocessor: 通过设置相似度分数阈值来过滤节点（因此仅嵌入式检索器支持）
#   -PrevNextNodePostprocessor: 基于 Node 关系，用额外的相关上下文增强检索到的 Node 对象。
"""  节点后处理器
https://docs.llamaindex.org.cn/en/stable/api_reference/postprocessor/
"""
#
### 配置响应合成
from llama_index.core.postprocessor import KeywordNodePostprocessor
node_postprocessors = [
    KeywordNodePostprocessor=(
        required_keywords=["key_a"],
        exclude_keywords=["key_b"],
    )
]
query_engine = RetrieverQueryEngine.from_args[
    retriever=retriever, 
    node_postprocessors=node_postprocessors,
]
response = query_engine.query("What did the author do growing up?")
# 检索器获取相关节点后，BaseSynthesizer 通过组合信息合成最终响应。
#
# - default：“创建并精炼”答案，通过依次处理每个检索到的 Node；这会为每个节点进行单独的 LLM 调用。适用于需要更详细的答案。
# - compact：在每次 LLM 调用期间“压缩”提示，将尽可能多的 Node 文本块填充到最大提示大小内。如果在一个提示中无法容纳太多块，则通过多个提示进行“创建并精炼”答案。
# - tree_summarize：给定一组 Node 对象和查询，递归构建一棵树并返回根节点作为响应。适用于摘要目的。
# - no_text：仅运行检索器以获取本应发送给 LLM 的节点，但不实际发送。然后可以通过检查 response.source_nodes 进行检查。响应对象将在第 5 节更详细地介绍。
# - accumulate：给定一组 Node 对象和查询，将查询应用于每个 Node 文本块，同时将响应累积到数组中。返回所有响应的连接字符串。适用于需要针对每个文本块单独运行相同查询的情况。
# - 结构化输出
# 
# 
# 后续：结构化输出、工作流

#### 4.结构化数据提取

In [ ]:

##### 1 使用结构化llms

from datetime import datetime 
from pydantic import BaseModel, Field 
from typing import List 
    
class LineItem(BaseModel):
    """ line item in a table-like data """
    item_name: str = Field(description="name of item ")
    price: float = Field(description="price of this item")
    
class Invoice(BaseModel):
    """ representation of information from a table-like data """
    invoice_id: str = Field(description="id")
    date: datetime = Field(description="date")
    line_items: List[LineItem] = Field(description="list of LineItem")
    

##### LLM做结构化信息提取, 
class Patent(BaseModel):
    """ patent """
    pass 

from llama_index.llms.huggingface import HuggingFaceLLM 

llm = HuggingFaceLLM(
    model_name     = r"Qwen/Qwen3-1.7B",
    tokenizer_name = r"Qwen/Qwen3-1.7B",
    context_window = 3900,  
    max_new_tokens = 640,
    generate_kwargs={"temperature": 0.7, "top_k": 30, "top_p": 0.95},
    device_map     ='cpu' 
)
# struct_llm = llm.as_structured_llm(Patent)
# response = struct_llm.complete(text)   
# ### response 是一个 LlamaIndex CompletionResponse 对象，它有两个属性： text 和 raw。 text 包含通过Pydantic摄取的响应的JSON序列化形式

# 结构化LLM的工作方式与常规LLM类完全相同：可以调用 chat、stream、achat、astream 等，并且在所有情况下，它都会以Pydantic对象响应。
# 还可以将结构化LLM作为参数传递给 VectorStoreIndex.as_query_engine(llm=sllm)，它将自动以结构化对象响应您的RAG查询。

##### 2 结构化信息预测
#
from llama_index.core.prompts import PromptTemplate 
prompt = PromptTemplate(
    "Extract an invoice from the following text. If you cannot find an invoice ID, use the company name '{company_name}' and the date as the invoice ID: {text}"
)



#### 5 跟踪和调试  todo

#### 6 评估  todo

#### 7 集成所有组件  todo

In [ ]:
# embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding = HuggingFaceEmbedding(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    max_length=1024,
    trust_remote_code=True,
    model_kwargs={"local_files_only": True},   # 允许联网 False , 禁止联网 True
    device='cpu',
)